In [1]:
# State the parameters of the pipeline
patient_list = [7]
disk = '/mnt/pre_epi_seizures/'
baseline_files = 'h5_files/processing_datasets/baseline_datasets_new'
seizure_files = 'h5_files/processing_datasets/seizure_datasets_new'
time_before_seizure = 50 * 60
time_after_seizure = 20 * 60
time_baseline = 4 * 60 * 60


In [2]:
import convertpandas as cv_pd
import pandas as pd
import sklearn.preprocessing as pp
import numpy as np

# Set parameters of analyssis
feature_slot = 'pca_beat_amp_computation'
patient_list = [3]
lead_list = ['ECG-']
scaler = pp.MinMaxScaler()
interim_processing = [scaler]

In [3]:
# Ingest Data
path_to_load = disk + seizure_files + '.h5'
path_to_map = disk + seizure_files + '_map.txt'
label_struct = {
                        'inter_ictal':{
                            'label': 'Inter-Ictal Data Points',
                            'color': 'blue',
                            'intervals_samples': [(1000 * 0 * 60, 1000 * 10 * 60 )]
                                     },

                        'pre_ictal':{
                            'label': 'Pre-Ictal data points',
                            'color': 'yellow',
                            'intervals_samples': [(1000 * 10 * 60, 1000 * 50 * 60 )]
                                     },
                        'ictal':{
                            'label': 'Ictal data points',
                            'color': 'red',
                            'intervals_samples': [(1000 * 50 * 60, 1000 * 55 * 60 )]
                                 },
                        }

seizure_data = cv_pd.convert_to_pandas(path_to_load, path_to_map,
                        patient_list, feature_slot,
                        lead_list, label_struct)

# set Labeling structure
path_to_load = disk + baseline_files + '.h5'
path_to_map = disk + baseline_files + '_map.txt'

baseline_label_struct = {
                         'baseline':{
                            'label': 'Baseine Data Points',
                            'color': 'green',
                            'intervals_samples': [(0, 1000 *  60 * 60 )]
                                    },
                        }

baseline_data = cv_pd.convert_to_pandas(path_to_load, path_to_map,
                        patient_list, feature_slot,
                        lead_list, baseline_label_struct)

[2018-03-15 21:44:35.748442]:get_signal:20:Loading [signal: 3_FA7774DK_ECG-_2017-04-20 07:38:44.000000_0][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_endparam$_/pca_beat_amp_computation_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_nr_comp:5_endparam$_]
[2018-03-15 21:44:35.749417]:get_signal:20:Loading [signal: 3_FA7774DR_ECG-_2017-04-20 16:09:45.000000_1][group: /3000_1200/raw_$beginwin_samplerate:1000_win:0.001_init:0_finish:4200_endwin$__$beginparams_param:None_endparam$_/baseline_removal_$beginwin_win:0.001__init:0__finish:4200__samplerate:1000_endwin$__$beginparam_filt:MedianFIR_endparam$_/rpeak_detection_$beginwin_win:rpeaks__samplerate:1000_endwin$__$beginparam_method:hamilton_end

closed
closed
closed
      eig_value_1  eig_value_2  eig_value_3   eig_value_4    eig_value_5  \
0        1.694293     6.458748    16.648878    106.122392   23094.766535   
1        1.076780     5.045120    11.038419    163.141151   23179.441892   
2        0.162152     5.131857    26.870452     60.130411   23201.964578   
3        0.261545     1.842108    18.795560     71.904513   22736.750491   
4        1.086293     3.973790    22.574858     84.829586   23255.145541   
5        1.167934     1.525366    28.963206     87.361172   23042.265056   
6        1.112480     2.128492    11.430741     36.648106   22407.797201   
7        0.985941     3.539536    12.343541     66.247517   22492.792401   
8        0.406264     3.089729     4.402463     69.607897   22942.910252   
9        0.631503     3.043285    10.877272     55.108026   23117.633460   
10       1.195617     3.874740    11.014302    100.937521   23933.344685   
11       0.530006     5.596810    10.688457     94.845056   24508.4

In [4]:
baseline_data = baseline_data.dropna(axis=0, how='any').reset_index(drop=True)

In [5]:
seizures = seizure_data['seizure_nr'].unique()
seizures

array([u'0', u'1', u'2'], dtype=object)

In [6]:
# randomly assign seizures
a = np.random.shuffle(seizures)
frac = len(baseline_data)/len(seizures)
for i, seizure in enumerate(seizures):
    baseline_data.loc[i*frac:(i+1)*frac]['seizure_nr'] = seizure
    
    

/home/franciscosargo/.virtualenvs/pre_epi_seizures/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [7]:
baseline_data

,eig_value_1,eig_value_2,eig_value_3,eig_value_4,eig_value_5,time_sample,patient_nr,seizure_nr,label,color
0,1.116552,3.013071,22.008130,48.798558,5058.286245,6542,3,1,Baseine Data Points,green
1,0.454303,2.916266,20.351146,48.332876,5574.426700,7478,3,1,Baseine Data Points,green
2,0.691106,5.628678,20.771745,32.179464,5357.242833,8403,3,1,Baseine Data Points,green
3,0.502401,3.588315,19.028960,37.396847,5170.059942,9327,3,1,Baseine Data Points,green
4,0.519193,3.629818,12.084851,40.451592,4652.830863,10213,3,1,Baseine Data Points,green
5,1.886049,3.053983,26.065330,77.767646,5371.119393,11091,3,1,Baseine Data Points,green
6,0.459124,5.628855,11.691472,159.783873,5519.246131,11998,3,1,Baseine Data Points,green
7,0.725779,5.695925,14.199516,134.020000,5355.517450,12982,3,1,Baseine Data Points,green
8,1.943985,5.857806,9.098195,70.316791,5214.663626,13984,3,1,Baseine Data Points,green
9,2.006842,3.088045,33.278509,85.646474,6231.522359,14968,3,1,Baseine Data Points,green
